# Import

In [1]:
from anytree import Node

In [2]:
with open('2.txt') as f:
    lines = f.readlines()

# Description of the data

In [3]:
len(lines)

1005

In [4]:
lines[0] #the number of vertices in the phenotype tree

'10000\n'

In [5]:
n = int(lines[0].rstrip())
n

10000

In [6]:
len(lines[1]) #the parent identifiers for the vertices

46035

In [7]:
parents = [int(i) for i in lines[1].rstrip().split(' ')]

In [8]:
parents[:5]

[1, 2, 3, 4, 1]

In [9]:
len(lines[2]) #information content values of the corresponding vertices

49991

In [10]:
info = [int(i) for i in lines[2].rstrip().split(' ')]

In [11]:
info[:5]

[223, 244, 989, 1581, 2530]

In [12]:
lines[3] #the number of diseases

'500\n'

In [13]:
m = int(lines[3].rstrip())
m

500

In [14]:
len(lines[4:m+4]) #contain descriptions of diseases

500

In [15]:
desc_d = []
for line in lines[4:m+4]:
    desc_d.append([int(i) for i in line.rstrip().split(' ')])

In [16]:
desc_d[:2]

[[6, 5915, 3457, 1699, 4876, 3799, 5605],
 [7, 3152, 4085, 9975, 7941, 9363, 6792, 3817]]

In [17]:
lines[m+4] #the number of patients

'500\n'

In [18]:
l = int(lines[m+4].rstrip())
l

500

In [25]:
len(lines[m+4+1:m+4+1+l]) #descriptions of patients

500

In [26]:
desc_p = []
for line in lines[m+4+1:m+4+1+l]:
    desc_p.append([int(i) for i in line.rstrip().split(' ')])

In [27]:
desc_p[:3]

[[1, 4138],
 [7, 6196, 1545, 4901, 144, 7886, 6982, 3777],
 [10, 1050, 5013, 3213, 7744, 9290, 692, 4233, 9361, 7570, 6421]]

In [29]:
lines[m+4+l]

'9 2507 1950 3374 990 4908 8045 5985 2410 5594\n'

# Create Tree Structure

## Initiate root

In [30]:
root = Node(1)

In [31]:
nodes = {}
nodes[root.name] = root

## Create Tree

In [32]:
%%time

for i, j in zip(range(2,n+1), parents):
    name = i
    nodes[name] = Node(name, parent=nodes[j])

CPU times: user 291 ms, sys: 20 ms, total: 311 ms
Wall time: 310 ms


In [33]:
import gc
gc.collect()

59

# Task

## Function to find LCA

In [34]:
def commonancestors(a, b):
    if a.parent == b:
        return b
    elif b.parent == a:
        return a
    ancestors = [node.ancestors for node in [a,b]]
    common = []
    for parentnodes in zip(*ancestors): 
        parentnode = parentnodes[0]
        if all([parentnode is p for p in parentnodes[1:]]):
            common.append(parentnode)
        else:
            break
    return tuple(common)

## IC dictionary

In [35]:
# create info dictionary
infod = {}

In [36]:
for i , j in enumerate(info):
    infod[i+1] = j

# Other variables

In [37]:
# disease score
d_scores = []

In [38]:
# result output
result = []

In [39]:
# create score
score = 0

In [40]:
# create minimum
maxim = 0

# Main code block

In [41]:
from tqdm import tqdm

In [42]:
%%time 

# loop through patients
for i in tqdm(range(1,l+1)):
    d_scores = []
    # loop through diseases
    for j in range(1, m+1):
        score = 0
        # loop through patient descriptions
        # skip first item, which is no. of desc
        for d in desc_p[i-1][1:]:
            maxim = 0
            # loop through disease descriptions:
            # skip first item, same reason as above
            for q in desc_d[j-1][1:]:
                # find LCA(q,d)
                anc = commonancestors(nodes[q],nodes[d])
                # anytree LCA function tweak
                try:
                    if len(anc) > 1:
                        key = anc[-1].name
                    elif len(anc) == 1:
                        key = anc[0].name
                except TypeError as e:
                    key = anc.name                        
                info_ = infod[key]
                # find max(IC(LCA(q,d)))
                if info_ >= maxim:
                    maxim = info_
            score += maxim
        d_scores.append(score)
    disease = d_scores.index(max(d_scores)) +1
    result.append(str(disease) + '\n')

100%|██████████| 500/500 [04:05<00:00,  2.04it/s]

CPU times: user 4min 4s, sys: 1.68 s, total: 4min 5s
Wall time: 4min 5s


In [43]:
with open('result2.txt', 'w') as f:
    f.writelines(result)

In [44]:
len(result)

500